# Quick Sample Flow - All Models Demo

This section demonstrates a quick sample run of all implemented model classes with minimal data and iterations to verify everything works correctly.

## Quick Setup - Load Minimal Data

In [1]:
import os
import sys
import numpy as np
import time
from copy import deepcopy
import random
import itertools
sys.path.append(os.path.join(os.path.curdir, ".."))
from framework.data_utils import load_cifar10_data
from models.cnn import TrainingConfig

# Add parent directory to path to import our models
sys.path.append(os.path.join(os.path.curdir, ".."))

# Dataset directories
DATASET_DIR = os.path.join(os.path.curdir, "..", ".cache", "processed_datasets")
CIFAR10_PATH = os.path.join(DATASET_DIR, "cifar10")

print("Quick sample flow setup completed!")
print(f"Dataset directory: {DATASET_DIR}")
print(f"CIFAR-10 path: {CIFAR10_PATH}")

Quick sample flow setup completed!
Dataset directory: .\..\.cache\processed_datasets
CIFAR-10 path: .\..\.cache\processed_datasets\cifar10


In [2]:
# Load datasets (small sample for quick demo)
import warnings
warnings.filterwarnings("ignore")

try:
    # Load datasets from cache
    cifar10_dataset = load_cifar10_data()

    # Take small samples for quick demo (100 samples each)
    SAMPLE_SIZE = 100

    # Sample from CIFAR-10
    cifar10_sample = (
        cifar10_dataset["train"].shuffle(seed=42).select(range(SAMPLE_SIZE))
    )
    cifar10_test_sample = cifar10_dataset["test"].shuffle(seed=42).select(range(50))

    # Extract a few samples to check shapes and pixel ranges
    sample_images = [np.array(img) for img in cifar10_sample['image'][:5]]  # First 5 images
    sample_labels = cifar10_sample['label'][:5]  # First 5 labels
    
    # Convert to numpy arrays for shape analysis
    sample_images_array = np.array(sample_images)
    sample_labels_array = np.array(sample_labels)
    
    # Observe Sample Shapes
    print(f"CIFAR-10 sample image shapes: {sample_images_array.shape}")  # (batch, height, width, channels)
    print(f"CIFAR-10 sample label shape: {sample_labels_array.shape}")   # (batch,)
    print(f"Individual image shape: {sample_images[0].shape}")           # (height, width, channels)
    
    # Observe Pixel Value Ranges
    pixel_min = np.min([np.min(img) for img in sample_images])
    pixel_max = np.max([np.max(img) for img in sample_images])
    print(f"CIFAR-10 pixel value ranges: [{pixel_min}, {pixel_max}]")
    
    # Show data types
    print(f"Image data type: {sample_images[0].dtype}")
    print(f"Label data type: {type(sample_labels[0])}") # Labels are encoded class ints
    
    print("Sample datasets loaded successfully!")
    print(
        f"CIFAR-10 sample: {len(cifar10_sample)} train, {len(cifar10_test_sample)} test"
    )

    # Aligning Variable Names with Tradition (for easier debugging)

    # Split into train and test sets from a HuggingFace Dataset
    X_train, y_train = sample_images, sample_labels
    X_test, y_test = [np.array(img) for img in cifar10_test_sample['image']], cifar10_test_sample['label']

except Exception as e:
    print(f"Error loading datasets: {e}")
    print("Please make sure datasets are downloaded and processed first.")

CIFAR-10 sample image shapes: (5, 32, 32)
CIFAR-10 sample label shape: (5,)
Individual image shape: (32, 32)
CIFAR-10 pixel value ranges: [0.0, 0.9672286510467529]
Image data type: float32
Label data type: <class 'int'>
Sample datasets loaded successfully!
CIFAR-10 sample: 100 train, 50 test


## Load All Model Classes

In [3]:
# Silence warnings to avoid printing full paths
import warnings
warnings.filterwarnings("ignore")

# Import all model classes
from models.decision_tree import DecisionTreeModel
from models.knn import KNNModel
from models.cnn import CNNModel


def create_sample_models():
    """Create instances of all model classes for testing"""
    models = {}

    # Decision Tree
    dt_model = DecisionTreeModel()
    dt_model.create_model()
    models["Decision Tree"] = dt_model
    print(f"Decision Tree model created: {dt_model.estimator}")

    # K-Nearest Neighbors
    knn_model = KNNModel()
    knn_model.create_model()
    models["K-Nearest Neighbors"] = knn_model
    print(f"KNN model created: {knn_model.estimator}")

    # CNN (Note: May require special handling due to PyTorch)
    try:
        cnn_model = CNNModel()
        cnn_model.create_model()
        models["Convolutional Neural Network"] = cnn_model
        print(f"CNN model created: {cnn_model.network}")
    except Exception as e:
        print(f"[ERROR] CNN model creation failed: {e}")
        print("CNN will be skipped in quick demo")

    return models


# Create model instances
sample_models = create_sample_models()

print("Model classes loaded successfully!")
for name, model in sample_models.items():
    print(f"  {name}: {type(model).__name__}")

print(f"\nTotal models available: {len(sample_models)}")

Decision Tree model created: DecisionTreeClassifier()
KNN model created: KNeighborsClassifier()
CNN model created: Backbone(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): AdaptiveAvgPool2d(output_size=1)
  )
  (fea

## Quick Hyperparameter Testing

In [4]:
def to_primitive(value):
    """Convert any complex to Python primitive types for sklearn compatibility"""
    if hasattr(value, 'item'):  # numpy scalar
        return value.item()
    elif isinstance(value, np.ndarray):
        return value.tolist()
    # Recursion for iterative data structures
    elif isinstance(value, (list, tuple)):
        return type(value)(to_primitive(v) for v in value)
    elif isinstance(value, dict):
        return {k: to_primitive(v) for k, v in value.items()}
    else:
        # Handle custom classes
        if hasattr(value, '__dict__'):
            return {k: to_primitive(v) for k, v in value.__dict__.items()}
        # Already a primitive
        return value

def convert_param_by_type(value, param_type):
    """Convert parameter value to correct Python type based on parameter type"""
    primitive_val = to_primitive(value)
    if param_type == "integer":
        return int(primitive_val)
    # Note: Learning Rate is often in a logarithmic scale rather than linear
    elif param_type == "float" or param_type == "float_log":
        return float(primitive_val)
    else:
        return primitive_val

def quick_hyperparameter_test(
    models_dict, X_train, y_train, X_test, y_test, dataset_name="Dataset"
):
    """Perform a quick hyperparameter test with limited iterations"""
    
    # Import data_utils functions for CNN support
    from framework.data_utils import create_dataloaders
    
    print(f"Starting quick hyperparameter test on {dataset_name}")
    print("=" * 60)

    results = {}
    # Default metric results (in case of an error)
    default_metrics = {
        "best_params": None,
        "best_score": -1.0,
        "metrics": {"accuracy": -1.0},
        "info": "Skipped in quick test - errornous run for current model"
    }

    for model_name, model in models_dict.items():
        print(f"\nTesting {model_name}...")

        # Check if model supports hyperparameter tuning
        if not hasattr(model, "get_param_space"):
            print(
                f"[WARNING] {model_name} does not support hyperparameter tuning. Using default params."
            )
            try:
                model_copy = deepcopy(model)
                
                # Handle CNN models with DataLoader
                if "Convolutional" in model_name or "CNN" in model_name:
                    try:
                        # Convert images to grayscale for CNN compatibility
                        from framework.data_utils import convert_to_grayscale
                        X_train_gray = [convert_to_grayscale(img) for img in X_train]
                        X_test_gray = [convert_to_grayscale(img) for img in X_test]
                        
                        # Use project's data_utils to create DataLoaders
                        train_loader, val_loader = create_dataloaders(
                            X_train_gray, y_train, X_test_gray, y_test, batch_size=32
                        )
                        # CNN expects train_loader, val_loader as arguments
                        model_copy.train(train_loader, val_loader)
                        
                        # Evaluate CNN model - models now handle ROC AUC errors internally
                        metrics = model_copy.evaluate(val_loader, y_test)
                    except Exception as cnn_error:
                        print(f"[ERROR] CNN training failed: {cnn_error}")
                        results[model_name] = {"error": f"CNN training failed: {str(cnn_error)[:50]}..."}
                        continue
                else:
                    # For sklearn models, flatten the images to 1D  
                    X_train_flat = np.array([img.flatten() for img in X_train])
                    X_test_flat = np.array([img.flatten() for img in X_test])
                    
                    model_copy.train(X_train_flat, y_train)
                    
                    # Evaluate sklearn models - models now handle ROC AUC errors internally
                    metrics = model_copy.evaluate(X_test_flat, y_test)

                results[model_name] = {
                    "best_params": "default",
                    "best_score": metrics.get("accuracy", default_metrics["metrics"]["accuracy"]),
                    "metrics": metrics,
                }
                # Observe the Accuracy from Metrics
                acc = metrics.get('accuracy', default_metrics["metrics"]["accuracy"])
                print(f"Default accuracy: {acc:.4f}")
            except Exception as e:
                print(f"[ERROR] {e}")
                results[model_name] = {"error": str(e)}
            continue

        # Get parameter space and sample a few combinations
        try:
            param_space = model.get_param_space()
            param_names = list(param_space.keys())

            # Extract actual values from ParamSpace objects
            def extract_param_values_for_quick_test(param_space):
                """Quick version of param extraction with fewer samples"""
                param_values = []
                param_types = []  # Track parameter types for proper conversion
                for param_name, param_def in param_space.items():
                    if hasattr(param_def, "param_type"):
                        # This is a ParamSpace object
                        param_type = param_def.param_type.value
                        param_types.append(param_type)
                        
                        def get_param_values_numeric(param_def):
                            """Helper to get numeric param values for quick test"""
                            values = []
                            if (
                                param_def.min_value is not None
                                and param_def.max_value is not None
                            ):
                                # Note: Learning Rate is often in a logarithmic scale rather than linear
                                # Handle logarithmic ranges for float_log type
                                if param_type == "float_log":
                                    # Use logarithmic spacing for log ranges
                                    log_values = np.logspace(
                                        np.log10(param_def.min_value), 
                                        np.log10(param_def.max_value), 
                                        3
                                    )
                                    values = [to_primitive(val) for val in log_values]
                                else:
                                    # Use linear spacing for regular ranges
                                    float_values = np.linspace(
                                        param_def.min_value, param_def.max_value, 3
                                    )
                                    values = [to_primitive(val) for val in float_values]
                            if (
                                param_def.default is not None
                                and param_def.default not in values
                            ):
                                values.append(to_primitive(param_def.default))
                            return values
                        
                        match param_type:
                            case "categorical":
                                values = param_def.choices
                            case "boolean":
                                values = [True, False]
                            case "integer":
                                # Sample a few integer values from the range for quick testing
                                values = get_param_values_numeric(param_def)
                                # Convert to integers explicitly
                                values = [int(val) for val in values] if values else [int(to_primitive(param_def.default or 1))]
                                
                                # Special handling for n_neighbors in KNN to prevent sample size issues
                                if param_name == "n_neighbors" and ("K-Nearest" in model_name or "KNN" in model_name):
                                    max_neighbors = max(1, len(X_train) // 3)  # Conservative limit
                                    values = [v for v in values if 1 <= v <= max_neighbors]
                                    if not values:  # If all filtered out, use safe default
                                        values = [min(3, max_neighbors)]
                                    print(f"[INFO] Limited n_neighbors to {values} (max: {max_neighbors})")          
                            case "float" | "float_log": # also cater for logarithmic learning rates
                                # Sample a few float values from the range for quick testing
                                # float_log uses logarithmic spacing, regular float uses linear spacing
                                values = get_param_values_numeric(param_def)
                                values = values if values else [to_primitive(param_def.default or 0.1)]
                            case _:
                                raise ValueError(f"Unsupported param type: {param_type}")
                        param_values.append(values)
                    else:
                        # This is already a list of values
                        param_values.append(param_def)
                        param_types.append(None)

                return param_values, param_types

            param_values, param_types = extract_param_values_for_quick_test(param_space)

            # Generate all combinations and sample a few
            all_combinations = list(itertools.product(*param_values)) # Cartesian Product
            max_test = min(3, len(all_combinations))  # Test max 3 combinations for speed
            test_combinations = (
                random.sample(all_combinations, max_test)
                if len(all_combinations) > max_test
                else all_combinations
            )

            print(f"Testing {len(test_combinations)}/{len(all_combinations)} parameter combinations...")

            best_score = -1.0
            best_params = None
            best_metrics = None

            # Prepare data once for efficiency
            if "Convolutional" in model_name or "CNN" in model_name:
                try:
                    # Convert images to grayscale for CNN compatibility
                    from framework.data_utils import convert_to_grayscale
                    X_train_gray = [convert_to_grayscale(img) for img in X_train]
                    X_test_gray = [convert_to_grayscale(img) for img in X_test]
                        
                    train_loader, val_loader = create_dataloaders(
                        X_train_gray, y_train, X_test_gray, y_test, batch_size=32
                    )
                    X_train_prep, y_train_prep = train_loader, val_loader
                    X_test_prep, y_test_prep = val_loader, y_test
                except Exception as dl_error:
                    print(f"[ERROR] DataLoader creation failed: {dl_error}")
                    results[model_name] = {"error": f"DataLoader failed: {str(dl_error)[:50]}..."}
                    continue
            else:
                X_train_prep = np.array([img.flatten() for img in X_train])
                X_test_prep = np.array([img.flatten() for img in X_test])
                y_train_prep, y_test_prep = y_train, y_test

            for i, param_combo in enumerate(test_combinations):
                current_params = dict(zip(param_names, param_combo))
                
                # Convert parameters with type awareness
                for param_name, (param_value, param_type) in zip(param_names, zip(param_combo, param_types)):
                    current_params[param_name] = convert_param_by_type(param_value, param_type)

                print(f"Combo {i + 1}: Testing params {current_params}")

                try:
                    # Create fresh model copy
                    model_copy = deepcopy(model)

                    # Set parameters - handle sklearn, PyTorch, and custom models
                    param_set_success = False
                    
                    if hasattr(model_copy, "estimator") and hasattr(model_copy.estimator, "set_params"):
                        # sklearn models (DecisionTree, KNN, etc.)
                        model_copy.estimator.set_params(**current_params)
                        param_set_success = True
                    elif "Convolutional" in model_name:
                        # CNN params will be handled via create_model then train
                        param_set_success = True
                    elif hasattr(model_copy, "set_params"):
                        # Models with direct set_params method
                        model_copy.set_params(**current_params)
                        param_set_success = True
                    elif hasattr(model_copy, "model") and hasattr(model_copy.model, "set_params"):
                        # Models with model attribute that has set_params
                        model_copy.model.set_params(**current_params)
                        param_set_success = True
                    
                    if not param_set_success:
                            print(f"Combo {i + 1}: ERROR - Model does not support parameter setting")
                            continue

                    # Train model based on type
                    if "Convolutional" in model_name:
                        # First create model with parameters, then train with DataLoaders
                        # Separate CNN-specific params from training config params
                        cnn_params = {}
                        training_config_params = {}
                        
                        for param_name, param_value in current_params.items():
                            if param_name in ['batch_size', 'learning_rate', 'optimizer', 'weight_decay']:
                                training_config_params[param_name] = param_value
                            else:
                                cnn_params[param_name] = param_value
                        
                        # Create model with CNN architecture params
                        model_copy.create_model(**cnn_params)
                        
                        # Create training config with training params
                        config = TrainingConfig(epochs=5, **training_config_params)
                        
                        # Train using the correct CNN signature: train(train_loader, val_loader, config=config)
                        model_copy.train(X_train_prep, X_test_prep, config=config)
                    else: 
                        model_copy.train(X_train_prep, y_train_prep)
                    
                    # Evaluate - models now handle ROC AUC errors internally
                    if "Convolutional" in model_name:
                        # For CNN, use the DataLoader for evaluation
                        metrics = model_copy.evaluate(X_test_prep, y_test)
                    else:
                        # sklearn models use flattened arrays
                        metrics = model_copy.evaluate(X_test_prep, y_test_prep)

                    current_score = metrics.get("accuracy", default_metrics["metrics"]["accuracy"])
                    print(f"Combo {i + 1}: Accuracy: {current_score:.4f}")

                    if current_score > best_score:
                        best_score = current_score
                        best_params = current_params.copy()
                        best_metrics = metrics.copy()

                except Exception as e:
                    print(f"Combo {i + 1}: ERROR - {e}...")
                    continue

            # Store results for this model
            results[model_name] = {
                "best_params": best_params,
                "best_score": best_score,
                "metrics": best_metrics or default_metrics["metrics"],
            }

            if best_params:
                print(f"Best params: {best_params}")
                print(f"Best score: {best_score:.4f}")
            else:
                print("No successful parameter combinations found!")

        except Exception as e:
            print(f"[ERROR] Failed to test hyperparameters for {model_name}: {str(e)[:80]}...")
            results[model_name] = {"error": str(e)}

    print("\n" + "=" * 60)
    print("Quick Hyperparameter Test Summary:")
    for model_name, result in results.items():
        if "error" in result:
            print(f"{model_name}: ERROR - {result['error']}")
        else:
            score = result.get("best_score", -1.0)
            print(f"{model_name}: Best Score = {score:.4f}")
    
    return results

In [ ]:
# Test on CIFAR-10 only
print("\n" + "=" * 60)
print("Testing all models on CIFAR-10 sample...")
cifar_results = quick_hyperparameter_test(
    sample_models, X_train, y_train, X_test, y_test, "CIFAR-10"
)

print("\n" + "=" * 80)
print("QUICK TEST RESULTS SUMMARY - CIFAR-10")
print("=" * 80)
for model_name, result in cifar_results.items():
    if "error" in result:
        print(f"[ERROR] {model_name} failed - {result['error']}")
    else:
        print(f"[RESULT] {model_name}: Accuracy = {result['best_score']:.4f}")


Testing all models on CIFAR-10 sample...
Starting quick hyperparameter test on CIFAR-10

Testing Decision Tree...
Testing 3/128 parameter combinations...
Combo 1: Testing params {'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion': 'entropy'}
Combo 1: Accuracy: 0.1600
Combo 2: Testing params {'max_depth': 11, 'min_samples_split': 11, 'min_samples_leaf': 5, 'criterion': 'entropy'}
Combo 2: Accuracy: 0.1600
Combo 3: Testing params {'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion': 'gini'}
Combo 3: Accuracy: 0.1600
Best params: {'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion': 'entropy'}
Best score: 0.1600

Testing K-Nearest Neighbors...
[INFO] Limited n_neighbors to [1] (max: 1)
Testing 3/4 parameter combinations...
Combo 1: Testing params {'n_neighbors': 1, 'weights': 'uniform', 'metric': 'manhattan'}
Combo 1: Accuracy: 0.0800
Combo 2: Testing params {'n_neighbors': 1, 'weights': 'distance', 'metric': 'minko

Train Loss: 2.3706, Train Acc: 0.0000 (0.00%)
Val Loss: 2.2940, Val Acc: 0.1200 (12.00%)
Saved best model (val_acc=0.1200) to .cache\models\cnn_cifar.pth

Epoch 2/5


Train Loss: 2.3705, Train Acc: 0.0000 (0.00%)
Val Loss: 2.2954, Val Acc: 0.1200 (12.00%)

Epoch 3/5


Train Loss: 2.3620, Train Acc: 0.0000 (0.00%)
Val Loss: 2.2965, Val Acc: 0.1200 (12.00%)

Epoch 4/5


Train Loss: 2.3544, Train Acc: 0.0000 (0.00%)
Val Loss: 2.2977, Val Acc: 0.1200 (12.00%)

Epoch 5/5


...

Train Loss: 2.4491, Train Acc: 0.0000 (0.00%)
Val Loss: 2.3043, Val Acc: 0.1000 (10.00%)
Saved best model (val_acc=0.1000) to .cache\models\cnn_cifar.pth

Epoch 2/5


Train Loss: 2.4415, Train Acc: 0.0000 (0.00%)
Val Loss: 2.3048, Val Acc: 0.1000 (10.00%)

Epoch 3/5


Train Loss: 2.3202, Train Acc: 0.0000 (0.00%)
Val Loss: 2.3045, Val Acc: 0.1200 (12.00%)
Saved best model (val_acc=0.1200) to .cache\models\cnn_cifar.pth

Epoch 4/5


Train Loss: 2.2365, Train Acc: 0.0000 (0.00%)
Val Loss: 2.3038, Val Acc: 0.0600 (6.00%)

Epoch 5/5


Train Loss: 2.2023, Train Acc: 0.0000 (0.00%)
Val Loss: 2.3029, Val Acc: 0.1000 (10.00%)

Training complete!
Best val acc: 0.1200 (12.00%)
Combo 3: Accuracy: 0.1000
Best params: {'kernel_size': 4, 'stride': 3, 'learning_rate': 0.00031622776601683794, 'batch_size': 16, 'weight_decay': 0.005, 'optimizer': 'SGD'}
Best score: 0.1000

Quick Hyperparameter Test Summary:
Decision Tree: Best Score = 0.1600
K-Nearest Neighbors: Best Score = 0.0800
Convolutional Neural Network: Best Score = 0.1000

QUICK TEST RESULTS SUMMARY - CIFAR-10
[RESULT] Decision Tree: Accuracy = 0.1600
[RESULT] K-Nearest Neighbors: Accuracy = 0.0800
[RESULT] Convolutional Neural Network: Accuracy = 0.1000
Combo 3: Accuracy: 0.1000
Best params: {'kernel_size': 4, 'stride': 3, 'learning_rate': 0.00031622776601683794, 'batch_size': 16, 'weight_decay': 0.005, 'optimizer': 'SGD'}
Best score: 0.1000

Quick Hyperparameter Test Summary:
Decision Tree: Best Score = 0.1600
K-Nearest Neighbors: Best Score = 0.0800
Convolutional Neu

Noticing the exceptionally low scores, it could be the reason of the **quick** test. It only uses 100 training samples to quickly verify whether the pipeline works normally. So now, we go ahead with a more detailed flow.

## Model Interface Verification

In [6]:
# Verify that all models implement the required interface correctly
def verify_model_interface(model_dict):
    """Verify that all models implement the BaseModel interface correctly"""
    print("Verifying model interfaces...")
    print("=" * 50)

    for name, model in model_dict.items():
        print(f"\n{name}:")

        # Check required methods
        required_methods = [
            "create_model",
            "get_param_space",
            "train",
            "predict",
            "evaluate",
        ]
        missing_methods = []

        for method in required_methods:
            if hasattr(model, method):
                print(f"{method}()")
            else:
                print(f"[WARNING] {method}() - MISSING")
                missing_methods.append(method)

        # Check parameter space
        try:
            if hasattr(model, "get_param_space"):
                param_space = model.get_param_space()
                print(f"Parameter space: {len(param_space)} parameters")
                for param_name, param_def in param_space.items():
                    print(f"- {param_name}: {param_def.param_type.value}")
            else:
                print("[WARNING] No parameter space available")
        except Exception as e:
            print(f"[WARNING] Parameter space error: {e}")

        # Check if model is created
        if hasattr(model, "model") and model.model is not None:
            print(f"Model instance: {type(model.model).__name__}")
        elif hasattr(model, "estimator") and model.estimator is not None:
            print(f"Model instance: {type(model.estimator).__name__}")
        elif hasattr(model, "network") and model.network is not None:
            print(f"Model instance: {type(model.network).__name__}")
        else:
            print("No model instance found")

        if missing_methods:
            print(f"[WARNING] INTERFACE INCOMPLETE: Missing {missing_methods}")
        else:
            print("[RESULT] INTERFACE COMPLETE")


# Run interface verification
verify_model_interface(sample_models)

Verifying model interfaces...

Decision Tree:
create_model()
get_param_space()
train()
predict()
evaluate()
Parameter space: 4 parameters
- max_depth: integer
- min_samples_split: integer
- min_samples_leaf: integer
- criterion: categorical
Model instance: DecisionTreeClassifier
[RESULT] INTERFACE COMPLETE

K-Nearest Neighbors:
create_model()
get_param_space()
train()
predict()
evaluate()
Parameter space: 3 parameters
- n_neighbors: integer
- weights: categorical
- metric: categorical
Model instance: KNeighborsClassifier
[RESULT] INTERFACE COMPLETE

Convolutional Neural Network:
create_model()
get_param_space()
train()
predict()
evaluate()
Parameter space: 6 parameters
- kernel_size: integer
- stride: integer
- learning_rate: float_log
- batch_size: categorical
- weight_decay: float
- optimizer: categorical
Model instance: Backbone
[RESULT] INTERFACE COMPLETE


## Quick Demo Summary

This quick sample flow demonstrates:

1. **All Model Classes Loaded**: Successfully imported and instantiated all 5 model classes from the `models/` directory
2. **Interface Compliance**: Verified that all models implement the required `BaseModel` interface
3. **Hyperparameter Testing**: Tested hyperparameter tuning functionality with small sample data
4. **Training & Evaluation**: Confirmed that all models can train and evaluate on both MNIST and CIFAR-10 data

### Models Tested:
- **Decision Tree Model** (`DecisionTreeModel`)
- **K-Nearest Neighbors Model** (`KNNModel`)
- **Convolutional Neural Network Model** (`CNNModel`)

This quick flow uses small sample sizes (100 training, 50 test samples) and limited hyperparameter combinations (max 3 per model) to ensure fast execution while still validating that the complete machine learning pipeline works correctly for all implemented model classes.

---

**Note**: For full experiments, use the comprehensive workflow sections below with complete datasets and extensive hyperparameter search.

## Model Interface Verification

# Establishing an Ordinary Model Training Workflow

## Load the Data

Let's try on 2 different image datasets - CIFAR-10 and MNIST. 

In [7]:
import os

DATASET_DIR = os.path.join(os.path.curdir, "..", ".cache", "processed_datasets")
CIFAR10_PATH = os.path.join(DATASET_DIR, "cifar10")
MNIST_PATH = os.path.join(DATASET_DIR, "mnist")

In [8]:
# Load from HuggingFace datasets
from datasets import load_from_disk

# Load all datasets from cache
cifar10_dataset = load_from_disk(CIFAR10_PATH)
mnist_dataset = load_from_disk(MNIST_PATH)

# Access train and test splits for all datasets
CIFAR10_TRAIN = cifar10_dataset["train"]
CIFAR10_TEST = cifar10_dataset["test"]
MNIST_TRAIN = mnist_dataset["train"]
MNIST_TEST = mnist_dataset["test"]

# Display dataset information
print("Loaded datasets from cache:")
print(f"CIFAR-10 Train: {len(CIFAR10_TRAIN):,} examples")
print(f"CIFAR-10 Test: {len(CIFAR10_TEST):,} examples")
print(f"MNIST Train: {len(MNIST_TRAIN):,} examples")
print(f"MNIST Test: {len(MNIST_TEST):,} examples")
print(f"\nCIFAR-10 classes: {CIFAR10_TRAIN.features['label'].names}")
print(f"MNIST classes: {list(range(10))}")  # MNIST has digits 0-9

Loaded datasets from cache:
CIFAR-10 Train: 50,000 examples
CIFAR-10 Test: 10,000 examples
MNIST Train: 60,000 examples
MNIST Test: 10,000 examples

CIFAR-10 classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
MNIST classes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## Prepare a Validation Set

In [9]:
# Split using HuggingFace datasets train_test_split method
cifar10_split = CIFAR10_TRAIN.train_test_split(test_size=500, seed=42)
CIFAR10_TRAIN = cifar10_split["train"]
CIFAR10_VAL = cifar10_split["test"]

mnist_split = MNIST_TRAIN.train_test_split(test_size=500, seed=42)
MNIST_TRAIN = mnist_split["train"]
MNIST_VAL = mnist_split["test"]

# Inspect the sizes
print("After splitting into Train and Validation sets:")
print(f"CIFAR-10 Train: {len(CIFAR10_TRAIN):,} examples")
print(f"CIFAR-10 Validation: {len(CIFAR10_VAL):,} examples")
print(f"MNIST Train: {len(MNIST_TRAIN):,} examples")
print(f"MNIST Validation: {len(MNIST_VAL):,} examples")

After splitting into Train and Validation sets:
CIFAR-10 Train: 49,500 examples
CIFAR-10 Validation: 500 examples
MNIST Train: 59,500 examples
MNIST Validation: 500 examples


## Get the Machine Learning Models

In [10]:
import sys

sys.path.append(os.path.join(os.path.curdir, ".."))


# Instantiate model classes (not the sklearn models directly)
dt_model = DecisionTreeModel()
dt_model.create_model()
knn_model = KNNModel()
knn_model.create_model()
cnn_model = CNNModel()
cnn_model.create_model()

# Display those model instances
models = {
    "Decision Tree Model": dt_model,
    "K-Nearest Neighbors Model": knn_model,
    "Convolutional Neural Network Model": cnn_model,
}

# Show the model classes
for name, model in models.items():
    print(f"{name}: {type(model).__name__}")

models

Decision Tree Model: DecisionTreeModel
K-Nearest Neighbors Model: KNNModel
Convolutional Neural Network Model: CNNModel


{'Decision Tree Model': <models.decision_tree.DecisionTreeModel at 0x19c01b04510>,
 'K-Nearest Neighbors Model': <models.knn.KNNModel at 0x19b8f2c3230>,
 'Convolutional Neural Network Model': <models.cnn.CNNModel at 0x19b8f2c3490>}

## Tune the Hyperparameters

We tune it with our own multi-objective fitness functions across different metaheuristics. For simlicity, let's just demonstrate how we weigh the importances of every metric.

In [11]:
# Define the extract_features_and_labels function to extract images and labels from HuggingFace datasets
def extract_features_and_labels(dataset, flatten_images=True, max_samples=None, convert_to_grayscale=False):
    """Extract features and labels from HuggingFace dataset
    
    Args:
        dataset: HuggingFace dataset with 'image' and 'label' columns
        flatten_images: Whether to flatten images to 1D arrays (for sklearn models)
        max_samples: Maximum number of samples to extract (None for all)
        convert_to_grayscale: Whether to convert images to grayscale (for CNN compatibility)
        
    Returns:
        tuple: (X, y) where X is image features and y is labels
    """
    # Import grayscale conversion utility
    from framework.data_utils import convert_to_grayscale
    
    # Limit samples if specified
    if max_samples is not None and len(dataset) > max_samples:
        dataset = dataset.select(range(max_samples))
    
    # Extract images
    X = []
    for img in dataset['image']:
        img_array = np.array(img)
        
        # Convert to grayscale if requested
        if convert_to_grayscale:
            img_array = convert_to_grayscale(img_array)
        
        if flatten_images:
            img_array = img_array.flatten()
        X.append(img_array)
    
    X = np.array(X)
    y = np.array(dataset['label'])
    
    return X, y

# Extract validation features and labels for CIFAR-10 (convert to grayscale for CNN compatibility)
X_CIFAR10_VAL, y_CIFAR10_VAL = extract_features_and_labels(
    CIFAR10_VAL, flatten_images=True, convert_to_grayscale=True
)
print("CIFAR-10 Validation (Grayscale):")
print(f"X_VAL shape: {X_CIFAR10_VAL.shape}")
print(f"y_VAL shape: {y_CIFAR10_VAL.shape}")
print(f"Data type: X={X_CIFAR10_VAL.dtype}, y={y_CIFAR10_VAL.dtype}")

# Extract validation features and labels for MNIST (already grayscale)
X_MNIST_VAL, y_MNIST_VAL = extract_features_and_labels(MNIST_VAL, flatten_images=True, convert_to_grayscale=True)
print("\nMNIST Validation (Grayscale):")
print(f"X_VAL shape: {X_MNIST_VAL.shape}")
print(f"y_VAL shape: {y_MNIST_VAL.shape}")
print(f"Data type: X={X_MNIST_VAL.dtype}, y={y_MNIST_VAL.dtype}")

# Show label examples
print(f"\nCIFAR-10 label examples: {y_CIFAR10_VAL[:5]}")
print(f"MNIST label examples: {y_MNIST_VAL[:5]}")
print(f"Pixel value range CIFAR-10: [{X_CIFAR10_VAL.min()}, {X_CIFAR10_VAL.max()}]")
print(f"Pixel value range MNIST: [{X_MNIST_VAL.min()}, {X_MNIST_VAL.max()}]")

CIFAR-10 Validation (Grayscale):
X_VAL shape: (500, 1024)
y_VAL shape: (500,)
Data type: X=float32, y=int64

MNIST Validation (Grayscale):
X_VAL shape: (500, 1024)
y_VAL shape: (500,)
Data type: X=float32, y=int64

CIFAR-10 label examples: [1 2 6 7 9]
MNIST label examples: [8 7 1 6 0]
Pixel value range CIFAR-10: [0.0, 1.0]
Pixel value range MNIST: [0.0, 1.0]


The cell below shows 3 alternative methods of extracting the images array and the labels array, just as a reference to check whether the above extraction behaves correctly. 

In [12]:
# Alternative methods for extracting X and y

# Method 1: Direct column access (simpler but less flexible)
print("=== Alternative Method 1: Direct Column Access ===")
y_cifar_simple = np.array(CIFAR10_VAL["label"])
print(f"y_CIFAR10_VAL shape: {y_cifar_simple.shape}")

# Method 2: Using dataset.to_pandas()
print("\n=== Alternative Method 2: Using to_pandas() ===")
cifar_df = CIFAR10_VAL.to_pandas()
print(f"DataFrame shape: {cifar_df.shape}")
print(f"DataFrame columns: {list(cifar_df.columns)}")

# Method 3: Batch processing for large datasets (memory efficient)
print("\n=== Alternative Method 3: Batch Processing ===")


def extract_in_batches(dataset, batch_size=1000, flatten_images=True):
    """Extract features and labels in batches to save memory"""
    total_samples = len(dataset)
    X_batches = []
    y_batches = []

    for i in range(0, total_samples, batch_size):
        batch = dataset[i : i + batch_size]

        # Process batch
        batch_images = []
        for img in batch["image"]:
            img_array = np.array(img)
            if flatten_images:
                img_array = img_array.flatten()
            batch_images.append(img_array)

        X_batches.append(np.array(batch_images))
        y_batches.append(np.array(batch["label"]))

    # Concatenate all batches
    X = np.vstack(X_batches)
    y = np.concatenate(y_batches)

    return X, y


# Example with small batch for demonstration
X_batch, y_batch = extract_in_batches(CIFAR10_VAL.select(range(100)), batch_size=50)
print(f"Batch extraction example - X shape: {X_batch.shape}, y shape: {y_batch.shape}")

=== Alternative Method 1: Direct Column Access ===
y_CIFAR10_VAL shape: (500,)

=== Alternative Method 2: Using to_pandas() ===
DataFrame shape: (500, 2)
DataFrame columns: ['image', 'label']

=== Alternative Method 3: Batch Processing ===
Batch extraction example - X shape: (100, 1024), y shape: (100,)
y_CIFAR10_VAL shape: (500,)

=== Alternative Method 2: Using to_pandas() ===
DataFrame shape: (500, 2)
DataFrame columns: ['image', 'label']

=== Alternative Method 3: Batch Processing ===
Batch extraction example - X shape: (100, 1024), y shape: (100,)
